# Computations of central derivatives of L-series

This notebook contains basic implementations of algorithms from the paper "Derivatives of L-series of Weakly Holomorphic Cusp Forms" by N. Diamantis and F. Strömberg. 

In particular we present an implementation of the formula 

$$\Lambda^{(m)}(f,k/2)=i^{2m-k/2}N^{k/4}\sum_{j=0}^{m}{m \choose j}\log^{j}\left(\frac{i}{\sqrt{N}}\right)\int_{i/\sqrt{N}}^{i/\sqrt{N}+1}f(z)\zeta^{(m-j)}\left(1-\frac{k}{2},z\right)dz\tag{*}$$

for computing the central value of the L-series $\Lambda^{(m)}(f,s)$ attached to a weakly holomorphic cusp form $f$. It should be noted that in case $f$ is holomorphic then the standard completed $L$-function of $f$ is  $L^{*}_{f}(s)=\Lambda^{(m)}(f,s)$.

In [ ]:
# A collection of basic functions that are needed 
import mpmath
from sage.modular.modform.element import Newform
from sage.rings.laurent_series_ring_element import LaurentSeries
from sage.rings.power_series_poly import PowerSeries_poly

def complex_coefficients(f, prec=30, bprec=53):
    """
    Get complex approximations of the Fourier series coefficients of f

    INPUT: 
    
    - `f` -- Newform or Laurentseries. In case of a Laurent series the input must have sufficiently many coefficients. 
    - `prec` -- integer - the number of coefficients to use.
    - `bprec` -- the number of bits of precision to use. 
    
    """
    CF = ComplexField(bprec)
    prec = ZZ(prec)    
    if isinstance(f,(LaurentSeries,PowerSeries_poly)):
        if f.prec() < prec:
            raise ValueError("Need to give input to higher precision!")
        coeffs = [c.complex_embedding(bprec) if hasattr(c,'complex_embedding') else CF(c) for c in f.coefficients()]
        exps = f.exponents()
    else:
        f.q_expansion(prec) 
        coeffs = [c.complex_embedding(bprec) if hasattr(c,'complex_embedding') else CF(c) for c in f.coefficients(prec)]
        if f.is_cuspidal():
            exps = range(1,prec+1)
    return dict(zip(exps,coeffs))


def findM_holoform(f,D, N=None, k=None):
    """
    Find a truncation precision M such that the holomorphic newform  f(x+iy) is approximated to 10^{-D} for 0<=x<=1.

    INPUT:
    
    - `f` -- Newform / q-expansion of cusp form
    - `D` -- integer
    
    """
    if isinstance(f,Newform):
        N = RR(f.level())
        k = RR(f.weight())
    elif not N or not k:
        raise ValueError("Need to give N or k.")
    sqrtN = N.sqrt()
    twopib = 2*RR.pi()/sqrtN
    Cbk = 4*RR(1).exp()*RR(k/2+1).gamma()*sqrtN/(2*RR.pi())
    RHS = (D*RR(10).log() + Cbk.log())/twopib
    f = k/2/twopib
    M0 = ceil(RHS)
    step = M0.log()*f
    for M in range(M0,2*M0):
        if M - RR(M).log()*f > M0:
            return M
    raise ArithmeticError("Could not find suitable M")
    
    
def findM_weakform(f,D,N,k):
    """
    Find a truncation precision M such that weakly modular cusp form  f(x+iy) is approximated to 10^{-D} for 0<=x<=1.
    Important note:  This is a heuristic formula only. 

    INPUT:
    
    - `f` -- Laurent expansion
    - `D` -- integer
    
    """
    N = RR(N)
    k = RR(k)
    sqrtN = N.sqrt()
    RHS = sqrtN*(D + N.log()) # Just assume all constants are 1(!)
    M0 = max(N,ceil(RHS))
    step = M0.sqrt()*N.sqrt()
    coeffs = complex_coefficients(f,prec=2*M0,bprec=53)
    eps = 2**(8-53)
    for M in range(M0,2*M0):
        if M - RR(M).sqrt()*N.sqrt() < M0:
            continue
        # Else check the difference of truncating here and a further point
        twopiix = 2*RR.pi()*CC(0,1)*0.3
        twopiy = 2*RR.pi()/sqrtN
        f1 = sum([c*CF(twopiix*n-twopiy*n).exp() for n,c in coeffs.items() if n < M])
        f2 = sum([c*CF(twopiix*n-twopiy*n).exp() for n,c in coeffs.items() if n < max(M+5,2*M0)])
        if abs(f1-f2) < eps:
            return M
    raise ArithmeticError("Could not find suitable M")
    
def get_parameters(f, k, N, prec=None, bprec=None, mpctx=None, is_holomorphic_cuspform=False):
    RF = RealField(bprec)
    D = ceil(bprec*RR(2).log()/RR(10).log())
    if prec:
        return D, prec
    if is_holomorphic_cuspform:        
        if prec is None:
            M = findM_holoform(f,D,k=k,N=N)
            prec = M + 1        
    else:
        if k is None or N is None:
            raise ValueError("Need to give k and N for weakly holomorphic modular forms.")
        factor = RF(1)
        if prec is None:
            M = findM_weakform(f,D,N,k)
            prec = M + 1
    return D, prec
    
def l_series_derivative_formula(f,m,k=None,N=None,prec=None,bprec=53,maxdegree=None,is_holomorphic_cuspform=False):
    """
    Use the formula (*) to compute the m-th derivative of the L-series Lambda(f,s) at s=k/2
    
    INPUT:
    
    - `f` -- function
    
    """
    RF = RealField(bprec)
    CF = ComplexField(bprec)
    if bprec == 53:
        mpctx = mpmath.fp
        mpmath.fp.prec=53
    else:
        mpctx = mpmath.mp
    mpmath.mp.prec=bprec
    if isinstance(f, Newform):
        is_holomorphic_cuspform = True
        k = f.weight()
        N = f.level()
    elif k is None or N is None:
            raise ValueError("Need to give k and N unless input is Newform")        
    
    D,prec = get_parameters(f,k,N,prec,bprec,mpctx, is_holomorphic_cuspform)
    if is_holomorphic_cuspform:
        factor = 1+CF(0,1)**(k - 2*m)
        if factor == 0:
            return mpctx.mpc(0,0)
    else:
        factor = 1
    if not maxdegree:
        if N < 1000:
            maxdegree = 7
        else:
            maxdegree = 10
    k_half = RF(k)/RF(2)
    coeffs = complex_coefficients(f,prec,bprec)
    sqrtN = RF(N).sqrt()
    ib = CF(0,1)/sqrtN
    b = sqrtN**-1    
    logib = ib.log()
    s= RF(1) - RF(k) / RF(2)
    twopi = RF.pi()*RF(2)
    twopii = CF(0,1)*twopi
    twopib = twopi*b
    coeffs_indexes = list(coeffs.keys())
    # pre-compute the y-parts
    vec = {n: coeffs[n]*RF(-twopib*n).exp() for n in coeffs_indexes}
    coeffs_indexes.sort(reverse=True)
    @cached_function
    def F(x):
        if x > 1/2:
            return F(1-x).conjugate()
        fval = 0
        twopiix = twopii*RF(x)
        for n in coeffs_indexes:
            fval += CF(twopiix*n).exp()*vec[n]
        return mpctx.mpc(fval.real(),fval.imag())
    @cached_function
    def hurwitz(x,r):
        z = mpctx.mpc(x,b)
        return mpctx.hurwitz(s,z,derivative=r)
    # Use the special formula for weight 2 and first derivative
    if k == 2 and m == 1:
        intfak = mpctx.log(sqrtN)-mpctx.mpc(0,1)*mpctx.pi/mpctx.mpf(2)
        @cached_function
        def wt2integrand(x):
            z = mpctx.mpc(x,b)
            return mpctx.log(mpctx.gamma(z)) + z*intfak
        def integrand(x): 
            f = F(x)
            z = wt2integrand(x)
            return f*z
        val,er = mpmath.mp.quad(integrand,[0,1],method='gauss-legendre', error=True,maxdegree=maxdegree,verbose=0)
        #print(val,er)
        return val*sqrtN*CF(0,1)*factor
    summa = 0
    for j in range(m+1):    
        def integrand(x): 
            f = F(x)
            z = hurwitz(x,m-j)
            return f*z
        val,er = mpmath.mp.quad(integrand,[0,1],method='gauss-legendre',error=True,maxdegree=maxdegree,verbose=0)
        #         print(val,er)
        term = CF(binomial(m,j))*val*logib**j
        summa += term
    f1 = CF(0,1)**(2*m - k_half)*RF(N)**(RF(k)/RF(4))
    return summa*f1*factor



## Examples for Holomorphic newforms

## Level 37 weight 2

In [ ]:
f1 =  Newforms(37,2,names='a')[0]
L1 = f1.lseries()
L1.derivative(1,1)*(RR(f1.level()).sqrt()/RR.pi()/2)**1

In [ ]:
l_series_derivative_formula(f1,1)

In [ ]:
%timeit L1.derivative(1,1)

In [ ]:
%timeit l_series_derivative_formula(f1,1)

## Level 127 weight 4

In [ ]:
f2 =  Newforms(127,4,names='a')[0]
L2 = f2.lseries()
print(f"L_f^*(2)={L2(2)}")
print(f"L_f^*'(2)={L2.derivative(2,1)*(RR(f2.level()).sqrt()/RR.pi()/2)**2}")
print(f"L_f^*''(2)={L2.derivative(2,2)*(RR(f2.level()).sqrt()/RR.pi()/2)**2}")

In [ ]:
l_series_derivative_formula(f2,0)

In [ ]:
l_series_derivative_formula(f2,1) # zero by symmetry

In [ ]:
l_series_derivative_formula(f2,2,bprec=60)

In [ ]:
%timeit L2.derivative(2,2)

In [ ]:
%timeit l_series_derivative_formula(f2,2,bprec=60)

## Level 5077 weight 2

In [ ]:
# Warning - this takes a long time
f3 = Newforms(5077,2,names='a')[0]
L3 = f3.lseries()

In [ ]:
print(f"L_f^*(1)={L3(1)}")
print(f"L_f^*'(1)={L3.derivative(1,1)*(RR(f3.level()).sqrt()/RR.pi()/2)**1}")
print(f"L_f^*''(1)={L3.derivative(1,2)*(RR(f3.level()).sqrt()/RR.pi()/2)**1}")
print(f"L_f^*'''(1)={L3.derivative(1,3)*(RR(f3.level()).sqrt()/RR.pi()/2)**1}")

In [ ]:
%timeit L3.derivative(1,3)

In [ ]:
l_series_derivative_formula(f3,0,bprec=53)

In [ ]:
l_series_derivative_formula(f3,1,bprec=53)

In [ ]:
l_series_derivative_formula(f3,2,bprec=53)

In [ ]:
l_series_derivative_formula(f3,3,bprec=60,prec=250)

In [ ]:
%timeit l_series_derivative_formula(f3,3,bprec=60,prec=250)

In [ ]:
15000/212.

In [ ]:
117.837959237940 - _

In [ ]:
%timeit l_series_derivative_formula(f3,3,bprec=53,prec=650)

In [ ]:
l=complex_coefficients(f3,1000,bprec=53)

## Examples for weak cusp forms

To construct weakly modular cusp forms we use eta functions as building blocks.
Recall that 
$$
\eta(\tau) = q^{\frac{1}{24}} \prod_{n\ge 1} \left( 1- q^n \right).
$$
If we define  
$$
\Delta^{+}_2 (\tau) = (\eta(\tau)\eta(2\tau))^8 = q - 8q^2 + 12q^3 + 64q^4 +O(q^5)$$ 
and 
$$j^{+}_{2}(\tau) = (\eta(\tau)/\eta(2\tau))^{24} + 24 + 2^{12}(\eta(2\tau)/\eta(\tau))^{24} = q^{-1} + 4372q + 96256q^2 + 1240002q^3 + O(q^4)$$
then it can be shown that $\Delta^{+}_2\in S_8(\Gamma_0(2))$ and $j^{+}_2\in S_0^{!}(\Gamma_0(2))$ are
both invariant under the Atkin-Lehner involution $W_2$. 

The following weakly holomorphic modular forms of weight $16$ on $\Gamma_0(2)$ were defined by  Choi and Kim [Weakly holomorphic Hecke eigenforms and Hecke eigenpolynomials, Advances in Mathematics 290 (2016)] 

- $f_{16,-2}(\tau) = \Delta^{+}_2(\tau)^2 = q^2 −16q^3 + O(q^4)$
- $f_{16,-1}(\tau) = \Delta^{+}_2(\tau)^2 (j_2^{+}(\tau) + 16) = q+4204q^3 +O(q^4)$
- $f_{16,0}(\tau) = \Delta^{+}_2(\tau)^2 (j_2^{+}(\tau)^2 + 16 j_2^{+}(\tau) - 8576)=1+261120q^3+O(q^4)$
- $f_{16,1}(\tau) = \Delta^{+}_2(\tau)^2 (j_2^{+}(\tau)^3 + 16 j_2^{+}(\tau)^2 - 12948j_2^{+}(\tau) - 427328)
 = q^{-1} +  7525650q^3 +O(q^4)$
- $f_{16,2}(\tau) = \Delta^{+}_2(\tau)^2 (j_2^{+}(\tau)^4 + 16 j_2^{+}(\tau)^3 - 17320 j_2^{+}(\tau)^2 - 593536j_2^{+}(\tau) - 27188524) = q^{-2} + 140479808q^3+O(q^4)$

and we see that $f_{16,-2},f_{16,-1}\in S_{16}^{+}(\Gamma_0(2))$ and $f_{16,1},f_{16,2}\in S_{16}^{!}(\Gamma_0(2))$ while $f_{16,0}$ is not cuspidal.

In [ ]:

from sage.modular.etaproducts import qexp_eta
def eta_quotient_q_expansion(exponents=[],arguments=[],prec=20):
    r"""
    Give the q-expansion of an eta quotient with given exponents and arguments.
    """
    eta = qexp_eta(ZZ[['q']],prec)
    R = eta.parent()
    q = R.gens()[0]
    res = R(1)
    prefak = 0
    for i in range(len(arguments)):        
        res = res*eta.subs({q:q**arguments[i]})**exponents[i]
        prefak = prefak+arguments[i]*exponents[i]
    return res,prefak/24

In [ ]:
prec = 200
quotient,prefak = eta_quotient_q_expansion(arguments=[1,2],exponents=[8,8],prec=prec)
q = quotient.parent().gen()
Delta2 = quotient*q**prefak
quo1,prefak1 = eta_quotient_q_expansion(arguments=[1,2],exponents=[24,-24],prec=prec)
quo2,prefak2 = eta_quotient_q_expansion(arguments=[1,2],exponents=[-24,24],prec=prec)
j2 = quo1*q**prefak1 + 24 + 2**12*quo2*q**prefak2

In [ ]:
f16 = {}
f16[-2] = Delta2**2
f16[-1] = Delta2**2*(j2+16)
f16[0] = Delta2**2*(j2**2+16*j2-8576)
f16[1] = Delta2**2*(j2**3+16*j2**2 - 12948*j2 - 427328)
f16[2] = Delta2**2*(j2**4+16*j2**3-17320*j2**2 - 593536*j2 +27188524)
k = 16
N = 2

Let's first check the formulas in the holomorphic cases $f_{16,-2}$ and $f_{16,-1}$.

Note first that the unique newform of level 2 and weight 16 is 
$$f = q - 128q^2 + 6252q^3 + 16384q^4 + 90510q^5 + O(q^6) = f_{16,-1} - 128f_{16,-2}$$


In [ ]:
f4=Newforms(2,16,names='a')[0]
L4=f4.lseries()
D = sqrt(2.)/(2*RR.pi())
print("L^*_f4=",L4(8)*(D)**8*RR(8).gamma())
# Note that since the value is not zero it is more complicated to calculate the derivative (even if we know it is zero...)
# We just do this to double check... 
d1 = L4.derivative(8)*(D)**8*RR(8).gamma()
d2 = L4(8)*( log(D)*D**8*RR(8).gamma()+D**8*RR(8).gamma()*psi(RR(8)))
print("L'^*_f4=",d1+d2)


In [ ]:
f4_alt=f16[-1]-128*f16[-2]
l_series_derivative_formula(f4_alt,0,k=16,N=2,bprec=53,is_holomorphic_cuspform=True)

We then check the computational error by comparing calculations with different precisions. 

In [ ]:
CF=ComplexField(100)

In [ ]:
CF(0.06298394789748197609,2.98342427489482e-17) - CF(128)*CF(0.00008045589767063483,5.88570627621543e-20)

In [ ]:
l_series_derivative_formula(f16[1],0,k=16,N=2,bprec=103)

In [ ]:
vals = {}
for bprec in [53,103,203]:
    print("prec=",bprec)
    vals[bprec] = {}
    for i in range(-2,3):
        vals[bprec][i] = {}
        if i == 0:
            continue
        for m in range(0,3):
            z = l_series_derivative_formula(f16[i],m,k=16,N=2,bprec=bprec,is_holomorphic_cuspform= i in [-2,-1])
            vals[bprec][i][m] = z
            print(f"L_(f_16,{i:>2})^({m}),8)={float(z.real):<35.32f} + {z.imag}i")


In [ ]:
for i in range(-2,3):
    if i == 0:
        continue
    for m in range(3):
        f1 = vals[53][i][m]
        f2 = vals[103][i][m]
        f3 = vals[203][i][m]
        print(f"diff2 = L_(f_16,{i:>2})^({m}),8) ={float(abs(f2-f3)):4.2e}")


In [ ]:
%timeit l_series_derivative_formula(f16[1],0,k=16,N=2,bprec=103)

In [ ]:
%timeit l_series_derivative_formula(f16[1],1,k=16,N=2,bprec=103)

In [ ]:
%timeit l_series_derivative_formula(f16[1],2,k=16,N=2,bprec=103)

In [ ]:
%timeit l_series_derivative_formula(f16[2],0,k=16,N=2,bprec=103)

In [ ]:
%timeit l_series_derivative_formula(f16[2],1,k=16,N=2,bprec=103)

In [ ]:
%timeit l_series_derivative_formula(f16[2],2,k=16,N=2,bprec=103)

## Extra checks

To verify the accuracy of the integral formula we also compare with the exponential sum evaluated using the analogue of the method by Buhler, Gross and Zagier as described in the paper.

In [ ]:
@cached_function()
def ksi_factors(j,l,n,prec=53):
    RF = RealField(prec)
    if j == 0:
        return RF(1)
    if l == n-j:
        return RF(l+1).gamma()/RF(n+1).gamma()
    if l > n - j:
        return 0
    if l == 0:
        return sum([RF(1)/RF(1+k)*ksi_factors(j-1,0,k) for k in range(n)])
    # if l > 0
    return sum([RF(1)/RF(k+l+j)*ksi_factors(j-1,l,k+l+j-1) for k in range(n-l-j+1)])
var('x')
pgamma = gamma(1+x)
gamma_taylor_coeffs = pgamma.taylor(x,0,10).coefficients(sparse=False)
def polP(r,t):
    """
    Evaluate the polynomial used in E_1^r
    """
    summa = 0
    CF = t.parent()
    for i in range(r+1):
        c = gamma_taylor_coeffs[r-i]
        summa += CF(c)*t**i/gamma(CF(i+1))
    return summa
    
def E1r(r,z,nmax=10000):
    """
    Evaluate E_1^{r}(z) 
    Note: we need to increase working precision to ensure that we can add all terms. 
    """
    summa = 0
    orig_prec = z.parent().prec()
    # Now see what we need in term of working precision... 
    # largest term ~ e^z and smallest are like z^(-r-1) e^-z
    prec = z.parent().prec()+3+2*abs(z)/RR(2).log() + 2*(r+1) * abs(z).log()
    prec = ceil(prec)
    eps = RR(2)**(-prec)
    CF = ComplexField(prec=prec)
    RF = RealField(prec=prec)
    z = RF(z)
    for n in range(1,nmax):
        term = (-1)**(n-r-1)/RF(n)**(r+1)/gamma(RF(n+1))*z**n
        summa += term
        if abs(term) < eps:
            break
    pol = polP(r+1,-z.log())
    return ComplexField(orig_prec)(pol+summa)
        
def Ej_minus_n(j,n,z,verbose=0):
    """
    Evaluate E^{(j)}_{-n}(z)
    
    """    
    s1 = 0
    RF = z.parent()
    for l in range(n-j+1):
        s1 += z**l/RF(l+1).gamma()*ksi_factors(j,l,n,prec=RF.prec())
    s1 = s1*(-z).exp()
    s2 = 0
    for m in range(1,j+1):
        term = ksi_factors(m-1,0,n,prec=RF.prec())*E1r(j-m,z) 
        if verbose>0:
            print(m,ksi_factors(m-1,0,n),'* E1r(',j-m,z,')(',E1r(j-m,z),')=',term)
        s2 += term
    f = RF(n+1).gamma()/z**(n+1)
    return f*(s1+s2)
    
def higher_derivative_using_sum(f,m,k=None,N=None,prec=200,bprec=53,verbose=0):
    RF = RealField(bprec)
    if k is None:
        k = f.weight()
    if N is None:
        N = f.level()
    if bprec == 53:
        mpctx = mpmath.fp
    else:
        mpctx = mpmath.mp
    if isinstance(f,Newform):
        factor = factorial(m)*(CF(0,1)**(k+2*m)+1)
    else:
        factor = factorial(m)
    if prec is None:
        D,prec = get_parameters(f,k,N,prec,bprec,mpctx, is_holomorphic_cuspform=False)
    coeffs = complex_coefficients(f,prec)
    sqrtN = RF(N).sqrt()
    two_pi_over_sqrtN = RF.pi()*RF(2)/sqrtN
    summa = 0
    s = k // 2 - 1
    coeff_indexes = list(coeffs.keys())
    coeff_indexes.sort(reverse=True)
    for n in coeff_indexes:
        c = coeffs[n]
        if c == 0 or n > prec:
            continue
        term = RF(c)*Ej_minus_n(m,s,two_pi_over_sqrtN*n,verbose=verbose-1)
        summa += term
        if verbose > 0:
            print(n,two_pi_over_sqrtN*n,Ej_minus_n(m,s,two_pi_over_sqrtN*n))
    return summa*factor


In [ ]:
for i in [1,2]:
    for m in [0,1,2]:
        f1 = higher_derivative_using_sum(f16[i],m=m,k=16,N=2,prec=100,bprec=103)
        f2 = vals[103][i][m]
        print(f"diff1 = L_(f_16,{i:>2})^({m}),8) ={float(abs(f1-f2)):4.2e}")

In [ ]:
for i in [1,2]:
    for m in [0,1,2]:
        f = higher_derivative_using_sum(f16[i],m=m,k=16,N=2,prec=100,bprec=103)
        print(f"L_(f_16,{i:>2})^({m}),8)={float(f.real()):<10.20f} + {f.imag()}i")

In [ ]:
%timeit higher_derivative_using_sum(f16[1],m=0,k=16,N=2,prec=100,bprec=103)

In [ ]:
%timeit higher_derivative_using_sum(f16[1],m=1,k=16,N=2,prec=100,bprec=103)

In [ ]:
%timeit higher_derivative_using_sum(f16[1],m=2,k=16,N=2,prec=100,bprec=103)

In [ ]:
%timeit higher_derivative_using_sum(f16[2],m=0,k=16,N=2,prec=100,bprec=103)

In [ ]:
%timeit higher_derivative_using_sum(f16[2],m=1,k=16,N=2,prec=100,bprec=103)

In [ ]:
%timeit higher_derivative_using_sum(f16[2],m=2,k=16,N=2,prec=100,bprec=103)

In [ ]:
%prun higher_derivative_using_sum(f16[2],m=2,k=16,N=2,prec=100,bprec=103)

In [ ]:
F = f16[2]*(1400258734875*I)/(29630464*RR.pi()**15)+f16[1]*(-374953/3776)+f16[-1]/256+f16[-2]

In [ ]:
m=1
CF=ComplexField(103)
vals[103][-2][m]*CF(0,1400258734875)/CF(29630464)/CF.pi()**15+vals[103][-1][m]*CF(-374953)/CF(3776)+vals[103][1][m]/CF(256)+vals[103][2][m]

In [ ]:
m=2
vals[103][1][m]*CF(16)/CF(75) + vals[103][2][m]*CF(4096)/CF(75)

In [ ]:
vals[103]